I. Import Packages and Datasets

In [1]:
#Importing functions library

import datetime, warnings, scipy 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from collections import OrderedDict
from matplotlib.gridspec import GridSpec
#from mpl_toolkits.basemap import Basemap
from sklearn import metrics, linear_model
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from scipy.optimize import curve_fit
plt.rcParams["patch.force_edgecolor"] = True
plt.style.use('fivethirtyeight')
mpl.rc('patch', edgecolor = 'dimgray', linewidth=1)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr"
pd.options.display.max_columns = 50
%matplotlib inline
warnings.filterwarnings("ignore")

In [2]:
#Importing Flight dataset as flights_df

flights_df = pd.read_csv("Flights.csv")

In [3]:
# Importing airport dataset as airports_df
airports_df = pd.read_csv("airports.csv")

In [4]:
# Importing airlines dataset as airlines_df
airlines_df = pd.read_csv("airlines.csv")

II. Data Wraggling for Flights Dataset

In [5]:
# Changing Dates and Times YEAR MOTNH DAY
flights_df["DATE"] = pd.to_datetime(flights_df[["YEAR",'MONTH','DAY']])

In [6]:
# Information on raw data 
print(flights_df.columns)

Index(['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'FLIGHT_NUMBER',
       'TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
       'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY', 'TAXI_OUT',
       'WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE',
       'WHEELS_ON', 'TAXI_IN', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME',
       'ARRIVAL_DELAY', 'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON',
       'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY',
       'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY', 'DATE'],
      dtype='object')


In [7]:
#Information on Column Types and No of Null Values
tab_info=pd.DataFrame(flights_df.dtypes).T.rename(index={0:'Column Type'})
tab_info=tab_info.append(pd.DataFrame(flights_df.isnull().sum()).T.rename(index={0:'null values (nb)'}))
tab_info=tab_info.append(pd.DataFrame(flights_df.isnull().sum()/flights_df.shape[0]*100).T.rename(index={0:'null values (%)'}))
tab_info

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,DATE
Column Type,int64,int64,int64,int64,object,int64,object,object,object,int64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,int64,float64,float64,int64,int64,object,float64,float64,float64,float64,float64,datetime64[ns]
null values (nb),0,0,0,0,0,0,7750,0,0,0,39515,39515,40229,40229,2,43071,43071,0,41296,41296,0,41296,43071,0,0,1008048,820047,820047,820047,820047,820047,0
null values (%),0,0,0,0,0,0,0.739098,0,0,0,3.76845,3.76845,3.83654,3.83654,0.000190735,4.10757,4.10757,0,3.9383,3.9383,0,3.9383,4.10757,0,0,96.135,78.2059,78.2059,78.2059,78.2059,78.2059,0


In [8]:
#Splitting data into 2 subsets - Arrival (arr_df) and Departure (dep_df)
arr_df = flights_df[['DATE','AIRLINE','ORIGIN_AIRPORT','DESTINATION_AIRPORT','SCHEDULED_ARRIVAL', "ARRIVAL_TIME", "ARRIVAL_DELAY", 'TAXI_OUT','WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'WHEELS_ON', 'TAXI_IN','DIVERTED','CANCELLED', 'CANCELLATION_REASON',
       'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY']]
display(arr_df.head(5))
dep_df = flights_df[['DATE','AIRLINE', 'ORIGIN_AIRPORT', "DESTINATION_AIRPORT",'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', "DEPARTURE_DELAY", 'TAXI_OUT','WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE',
       'WHEELS_ON', 'TAXI_IN','DIVERTED', 'CANCELLED', 'CANCELLATION_REASON','AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY','LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY']]
display(dep_df.head(5))


,DATE,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015-01-01,AS,ANC,SEA,430,408.0,-22.0,21.0,15.0,205.0,194.0,169.0,1448,404.0,4.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-01-01,AA,LAX,PBI,750,741.0,-9.0,12.0,14.0,280.0,279.0,263.0,2330,737.0,4.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-01-01,US,SFO,CLT,806,811.0,5.0,16.0,34.0,286.0,293.0,266.0,2296,800.0,11.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-01-01,AA,LAX,MIA,805,756.0,-9.0,15.0,30.0,285.0,281.0,258.0,2342,748.0,8.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015-01-01,AS,SEA,ANC,320,259.0,-21.0,11.0,35.0,235.0,215.0,199.0,1448,254.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


,DATE,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015-01-01,AS,ANC,SEA,5,2354.0,-11.0,21.0,15.0,205.0,194.0,169.0,1448,404.0,4.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-01-01,AA,LAX,PBI,10,2.0,-8.0,12.0,14.0,280.0,279.0,263.0,2330,737.0,4.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-01-01,US,SFO,CLT,20,18.0,-2.0,16.0,34.0,286.0,293.0,266.0,2296,800.0,11.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-01-01,AA,LAX,MIA,20,15.0,-5.0,15.0,30.0,285.0,281.0,258.0,2342,748.0,8.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015-01-01,AS,SEA,ANC,25,24.0,-1.0,11.0,35.0,235.0,215.0,199.0,1448,254.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


Feature Selection

The following features are removed from the dataset: 
    - Flight Number: As this is unable to isolate and indicate the reason for the delay - aircraft, airports, etc but only illustrates an instance of the delay
    - Tail Number: While the tail number indicates a particular plane, more concrete details on the model is required to justify delays attributed to flight mechanics
    

In [9]:
#Replace the attribute values for cancellation reasons 
arr_df = arr_df.replace("A", "Airline/Carrier")
arr_df = arr_df.replace("B", "Weather")
arr_df = arr_df.replace("C", "National Air System")
arr_df = arr_df.replace("D", "Security")

dep_df = dep_df.replace("A", "Airline/Carrier")
dep_df = dep_df.replace("B", "Weather")
dep_df = dep_df.replace("C", "National Air System")
dep_df = dep_df.replace("D", "Security")


In [10]:
# Removing the rows with null values (about 3% or 4% of all datapoint)
# Remove rows with missing "ARRIVAL_TIME" & "ARRIVAL_DELAY" from arr_df
arr_df = arr_df.dropna(axis = 0 , subset = ['ARRIVAL_TIME', 'ARRIVAL_DELAY'])

#Data Cleaning for Flights for dep_df
dep_df = dep_df.dropna(axis = 0 , subset = ['DEPARTURE_TIME', 'DEPARTURE_DELAY'])


In [11]:
#Change the attribute of of airport to actual airport name

#Setting the airports into a dictionary of IATA code to Airport Name
abbr_airports = airports_df.set_index('IATA_CODE')['AIRPORT'].to_dict()
abbr_airports

# Reassigning the values of destination and origin airport codes with its actual name
arr_df = arr_df.replace({"DESTINATION_AIRPORT":abbr_airports})
arr_df = arr_df.replace({"ORIGIN_AIRPORT": abbr_airports})
dep_df = dep_df.replace({"DESTINATION_AIRPORT": abbr_airports})
dep_df = dep_df.replace({"ORIGIN_AIRPORT": abbr_airports})

In [12]:
#Change the attribute of of airlines to actual airlines name

# Creating dictionary on the airline code and company
abbr_companies = airlines_df.set_index('IATA_CODE')['AIRLINE'].to_dict()
abbr_companies

# Replacing IATA Airline code with the actual airline name
arr_df = arr_df.replace({"AIRLINE":abbr_companies})
dep_df = dep_df.replace({"AIRLINE": abbr_companies})

Data Cleaning
- Missing Values Manipulation: as the rows with null values only represent 3% - 4% of all datapoints, we have decided to remove the rows with null values
- Attribute vluaes for cancellation reasons are included to give more contextual meaning to the attribute values 

In [13]:
display(dep_df.head(3))
display(arr_df.head(3))

,DATE,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015-01-01,Alaska Airlines Inc.,Ted Stevens Anchorage International Airport,Seattle-Tacoma International Airport,5,2354.0,-11.0,21.0,15.0,205.0,194.0,169.0,1448,404.0,4.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-01-01,American Airlines Inc.,Los Angeles International Airport,Palm Beach International Airport,10,2.0,-8.0,12.0,14.0,280.0,279.0,263.0,2330,737.0,4.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-01-01,US Airways Inc.,San Francisco International Airport,Charlotte Douglas International Airport,20,18.0,-2.0,16.0,34.0,286.0,293.0,266.0,2296,800.0,11.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


,DATE,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015-01-01,Alaska Airlines Inc.,Ted Stevens Anchorage International Airport,Seattle-Tacoma International Airport,430,408.0,-22.0,21.0,15.0,205.0,194.0,169.0,1448,404.0,4.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-01-01,American Airlines Inc.,Los Angeles International Airport,Palm Beach International Airport,750,741.0,-9.0,12.0,14.0,280.0,279.0,263.0,2330,737.0,4.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-01-01,US Airways Inc.,San Francisco International Airport,Charlotte Douglas International Airport,806,811.0,5.0,16.0,34.0,286.0,293.0,266.0,2296,800.0,11.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# Adding boolean column: Delay = 1, No delay (Early/Punctual) = 0
#cond_early_arr = arr_df["ARRIVAL_DELAY"] < 0
#cond_punctual_arr = arr_df['ARRIVAL_DELAY'] == 0
arr_df["DELAY"] = np.where(arr_df['ARRIVAL_DELAY'] <= 0, "0", "1")
arr_df.head()

# Adding boolean column: Delay = 1, No delay (Early/Punctual) = 0
#cond_early_arr = arr_df["ARRIVAL_DELAY"] < 0
#cond_punctual_arr = arr_df['ARRIVAL_DELAY'] == 0
dep_df["DELAY"] = np.where(dep_df['DEPARTURE_DELAY'] <= 0, "0", "1")
dep_df.head()

,DATE,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,DELAY
0,2015-01-01,Alaska Airlines Inc.,Ted Stevens Anchorage International Airport,Seattle-Tacoma International Airport,5,2354.0,-11.0,21.0,15.0,205.0,194.0,169.0,1448,404.0,4.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0
1,2015-01-01,American Airlines Inc.,Los Angeles International Airport,Palm Beach International Airport,10,2.0,-8.0,12.0,14.0,280.0,279.0,263.0,2330,737.0,4.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0
2,2015-01-01,US Airways Inc.,San Francisco International Airport,Charlotte Douglas International Airport,20,18.0,-2.0,16.0,34.0,286.0,293.0,266.0,2296,800.0,11.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0
3,2015-01-01,American Airlines Inc.,Los Angeles International Airport,Miami International Airport,20,15.0,-5.0,15.0,30.0,285.0,281.0,258.0,2342,748.0,8.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0
4,2015-01-01,Alaska Airlines Inc.,Seattle-Tacoma International Airport,Ted Stevens Anchorage International Airport,25,24.0,-1.0,11.0,35.0,235.0,215.0,199.0,1448,254.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0


Feature Re-engineering

To count the instances of delays - whether the flight was delayed (1) or not delayed (0), we added another column of binary option. 
Hence if: 
    - the actual departure time > scheduled departure time 
    - or actual arrival time > scheduled arrival time 

Then, 1 will be assigned to departure delay or arrival delay respectively. 
This then tells us the proportion of arrival and departure delays relative to the entire population.

In [19]:
# Exporting cleaned datafile into csv

arr_df.to_csv('outputarr_df.csv', encoding='utf-8')
dep_df.to_csv('outputdep_df.csv', encoding='utf-8')

